# Flask: повторение

## Структура проекта
В среднем, структура проекта на Flask выглядит так *(и должна выглядеть так)*:
```
my_app
    templates
    static
        img
        style.css
    myapp.py
```

## Процентная кодировка
*Сумасшедший кот очень смешно 3D*

— результат Яндекса по запросу "котики"

Например, если искать в Яндексе, то ссылка, которая вернётся, будет иметь вид `yandex.ru/search/?msid=(какие-то_цифры)&text=(процентная_кодировка)`. Это происходит, потому что в сроке не может быть символов Юникода. 

Для того, чтобы перекодировать любой текст в процентную кодировку, существуют специальные функции в модуле urllib.parse:

In [ ]:
from urllib.parse import quote, unquote

Функция `quote(строка)` вернёт строку, закодированную в процентной кодировке, `unquote()` — из процентной кодировки сделает юникодную строку.

In [ ]:
>>> quote('котик')
'%D0%BA%D0%BE%D1%82%D0%B8%D0%BA'
>>> unquote('%D0%BA%D0%BE%D1%82%D0%B8%D0%BA')
'котик'

## Собственный корпус
Структура проекта/папки:
```
corpus
    templates
        index.html
    data
        test.txt
    corpus-code.py
```

В качестве текста для корпуса — Тысяча и одна ночь с сайта Гутенберга (https://www.gutenberg.org/files/34206/34206-0.txt).

Минимальный код для запуска хотя бы чего-то на фласке:

In [ ]:
from flask import Flask
from flask import render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)

При этом index.html выглядит так:
```html
<html>
	<head>
		<title>Корпус</title>
	</head>
	<body>
		<h1>Добро пожаловать в корпус "Тысяча и одна ночь!"!/h1>
	</body>
</html>
```

Добавляем функции в код. Во-первых, должна быть **страница results**, на которую пользователь будет попадать, если он что-то введёт в форму на главной странице. Если нет, то программа отправляет его обратно на index.

In [ ]:
# Добавляем нужные модули и функции
from flask import render_template, redirect, url_for, request
from urllib.parse import unquote

# Сама функция
@app.route('/results')
def result(request):
    if request.args:
        word = unquote(request.args['word'])
        sentences = search(word)
        return render_te('results.html', sents=sentences)
    return redirect(url_for('index'))

Во-вторых, нужно где-то искать, значит, нужно **"собрать" корпус из текста**. Функция `collect_corpus()` берёт текст, читает его по строчкам, удаляет пробелы в конце и записывает строчку в массив, если она не пустая. Полученный массов строчек возвращается. Этот массив записывается в переменную corpus, которую мы будем хранить в главной функции после `if __name__ == '__main__'`

In [ ]:
def collect_corpus():
    with open('data/test.txt', 'r', encoding='utf-8') as f:
        sents = [line.strip() for line in f.readlines()
                if line.strip != '']
    return sents

# Добавляем переменную с корпусом в главную функцию:
if __name__ == '__main__':
    corpus = collect_corpus()
    app.run(debug=True)

### Отступление: list comprehensions
(когда-нибудь будет)
___

Ещё нужна **функция поиска** (здесь тоже используются list comprehesions):

In [ ]:
def search(word):
    return [sent for sent in corpus if word in sent]

По пути верстаем **страницу с результатами results.html**, в ней используются последовательности в фигурных скобках:

* `{%%}` используется для цикла (который надо обязательно закрыть при помощи `{% endfor %}`, в отличие от питона
* `{}` — для вывода переменной
```html
<html>
	<head>
		<title>Корпус</title>
	</head>
	<body>
		<h1>Результаты</h1>
		<ol>
		{% for sent in sents %}
			<li> {{ sent }} </li>
		{% endfor %}
		</ol>
	</body>
</html>
```

### Прикручиваем дополнительные плюшки
Выделим **результат поиска жирным**. Для этого немного дополним код results.html:
```html
{% for sent in sents %}
			<li> {{ sent | safe }} </li>
		{% endfor %}
```
Такая последовательность говорит фласку, что всё ок, на месте sent будет нормальный html-код, который можно будет нормально скомпилировать.

После этого — поправим код в функции поиска:

In [ ]:
def search(word):
    return [sent.replace(word, '<b>' + word + '</b>') for sent in corpus if word in sent]

Функция `sent.replace(word, '<b>' + word + '</b>')` меняет просто вхождения word на вхождения word в теге <b>, т.е. делает их жирными.

### Код и результаты работы
Здесь же в папке 2016-11-07 corpus:
* corpus_code.py — сам код,
* templates — html-коды страниц
* папка data, в которой test.txt — исходный текст, по которому мы ищем